In [ ]:
from IPython import get_ipython
from IPython.display import display
# %%
# IMPORTANTE: EJECUTA ESTA CELDA PARA IMPORTAR TUS FUENTES DE DATOS DE KAGGLE,
# LUEGO PUEDES ELIMINAR ESTA CELDA SI LO DESEAS.
# NOTA: ESTE ENTORNO DE NOTEBOOK ES DIFERENTE AL ENTORNO DE PYTHON DE KAGGLE,
# POR LO QUE PODRÍA FALTAR ALGUNA LIBRERÍA USADA EN TU NOTEBOOK.
import kagglehub
yudhaislamisulistya_plants_type_datasets_path = kagglehub.dataset_download('yudhaislamisulistya/plants-type-datasets')
keras_efficientnetv2_keras_efficientnetv2_b0_imagenet_2_path = kagglehub.model_download('keras/efficientnetv2/Keras/efficientnetv2_b0_imagenet/2')

print('Importación de fuente de datos completada.')

## Descripción del dataset: 
El dataset contiene 30 000 imágenes de plantas, con 1000 imágenes por clase, y una colección diversa de 30 clases de plantas y 7 tipos de plantas, incluyendo cultivos, frutales, plantas industriales, medicinales, frutos secos, tubérculos y hortalizas. Las 30 clases de plantas incluyen especies populares como el plátano, el coco y la piña, así como plantas menos conocidas como el bilimbi y el galanga.

## Preparación de datos
El primer paso incluye:

Importación de bibliotecas
Creación de conjuntos de datos
En la carpeta "plants-type-datasets" que se encuentra en la plataforma de kaggle hay 30 000 imágenes de plantas divididas en tres carpetas (entrenamiento, prueba y validación). Para nuestra notebook, las tres carpetas se unirán y dividirán aleatoriamente más adelante. Por lo tanto, al principio habrá pasos adicionales para la preparación de los conjuntos de datos.

In [ ]:
import os
import random
import pandas as pd
import numpy as np
from IPython.display import Image

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# TensorFlow
import tensorflow as tf

# keras_cv para el uso del modelo preentrenado
import keras_cv

In [ ]:
# Preparación de etiquetas y rutas de imágenes
dataset_root = '/kaggle/input/plants-type-datasets/split_ttv_dataset_type_of_plants'
labels = []
images_path = []

# Iterar a través de los subdirectorios (categorías de plantas)
for dataset in os.listdir(dataset_root):
    for plant_category in os.listdir(os.path.join(dataset_root, dataset)):
        if os.path.isdir(os.path.join(dataset_root, dataset, plant_category)):
            images = os.listdir(os.path.join(dataset_root, dataset, plant_category))

            # Crea un marco de datos con image_path y etiqueta
            for image in images:
                image_path = os.path.join(dataset_root, dataset, plant_category, image)
                images_path.append(image_path)
                labels.append(plant_category)

In [ ]:
# verificación de la etiqueta y la ruta de la imagen
i = random.randint(0, 30000)
print(f'number of labels: {len(labels)} and images {len(images_path)}\n')
print('label of random plant: ', labels[i])
print('path to random image: ', images_path[i], '\n')
Image(images_path[i])
# preparación general de X e y
X = images_path
y = labels

# División del dataset para entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1)

len(X_train), len(y_train), len(X_val), len(y_val), len(X_test), len(y_test)

# Preprocesamiento de imágenes
## (conversión de imágenes en tensores)
Pasos para convertir imágenes a formato numérico y dividirlas en fragmentos más pequeños (lotes)

In [ ]:
# parámetros para la imagen y el tamaño del lote
img_size = 28
batch_size = 32 

def prepare_image(image_path):
    '''
    turns an image into a tensor
    '''
    # leer una imagen
    image = tf.io.read_file(image_path)
    # convertir una imagen a versión numérica
    image = tf.image.decode_jpeg(image, channels=3)
    # convierte colores de 0-255 a 0-1
    image = tf.image.convert_image_dtype(image, tf.float32)
    # cambiar el tamaño
    image = tf.image.resize(image, size=[img_size, img_size])

    return image

def get_label_image(image_path, label):
    image = prepare_image(image_path)

    return image, label

def create_batches(X, y=None, batch_size=batch_size, test_data=False):
    '''
    split a dataset to batches
    '''
    if test_data:
        data = tf.data.Dataset.from_tensor_slices((tf.constant(X), tf.constant(y)))
        data_batch = data.map(get_label_image).batch(batch_size)
    else:
        data = tf.data.Dataset.from_tensor_slices((tf.constant(X), tf.constant(y)))
        data = data.shuffle(buffer_size=len(X))
        data_batch = data.map(get_label_image).batch(batch_size)

    return data_batch

# Creación de lotes para todos los conjuntos de datos

# Entrenamiento de datos
train_data = create_batches(X_train, y_train)

# datos de prueba
test_data = create_batches(X_test, y_test, test_data=True)

# datos de validación
val_data = create_batches(X_val, y_val, test_data=True)

# Creación, entrenamiento y validación del modelo

En este capítulo, construiremos y entrenaremos los siguientes modelos:
* Modelo preentrenado del repositorio de Kaggle: https://www.kaggle.com/models/keras/efficientnetv2



In [ ]:
# declaración de formas de entrada y salida
input_shape = [img_size, img_size, 3]
output_shape = len(encoding_labels)

## Modelo 
As a model  we will use one of the earlier trained model from kaggle repository: **EfficientNetV2**

https://www.kaggle.com/models/keras/efficientnetv2

In [ ]:
# definir el modelo
model = keras_cv.models.ImageClassifier.from_preset(
    "efficientnetv2_b0_imagenet",
    num_classes=output_shape
)

# compilar el modelo
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

# construir el modelo
model.build(input_shape)

model.summary()

# entrenamiento de modelos
model.fit(x=train_data,
            epochs=5,
            validation_data=val_data,
            validation_freq=1
         )

# Guarda model
model.save('modelos/aprendiendomachin.h5')

## Predicciones (opcional usarlo)

In [ ]:
# función para codificar etiquetas
def pred_labels(prediction_propabilities):

    return encoding_labels[np.argmax(prediction_propabilities)]

# verificación de modelos en datos de prueba
models = [ model]
index = random.randint(0, len(y_val))

for model in models:
    print(model)
    y_pred = model.predict(test_data)
    predictions = []
    test = []
    for i in range(len(y_pred)):
        predictions.append(pred_labels(y_pred[i]))
        test.append(pred_labels(y_test[i]))

    label = pred_labels(y_pred[index])
    print(f'Is {pred_labels(y_test[index])} predicted properly? Prediction: {label}. So:  {label==pred_labels(y_test[index])}')
    print('Accuracy for the whole test dataset: ', round(accuracy_score(test, predictions), 2), '\n--- ---\n')

print('\n\n')
print('Random plant\'s picture from test dataset which name models tried to predict')
Image(X_test[index])